# 09: FastAPI Analytics Service Integration Demo

**Date:** 2026-02-04  
**Author:** IBM Bob  
**Status:** Active

## Overview

This notebook demonstrates integration with the **FastAPI Analytics Service**, which provides RESTful endpoints for:

1. **TBML Detection**: Trade-Based Money Laundering pattern detection
2. **Insider Trading Detection**: Coordinated trading and communication analysis
3. **UBO Discovery**: Ultimate Beneficial Owner identification
4. **Graph Statistics**: Real-time graph analytics

---

## Setup and Configuration

In [ ]:
# Standard notebook initialization
import sys
from pathlib import Path

from notebook_config import init_notebook

config = init_notebook(check_env=True, check_services=False)
PROJECT_ROOT = config['project_root']
print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time

API_BASE_URL = "http://localhost:8001"
print(f"API Base URL: {API_BASE_URL}")

## 1. Health Check

In [ ]:
def check_api_health():
    try:
        response = requests.get(f"{API_BASE_URL}/health", timeout=5)
        if response.status_code == 200:
            print("API Service is healthy")
            return True
        return False
    except requests.exceptions.ConnectionError:
        print("Cannot connect to API. Start with: uvicorn src.python.api.main:app --reload")
        return False

api_healthy = check_api_health()

## 2. Graph Statistics

In [ ]:
if api_healthy:
    response = requests.get(f"{API_BASE_URL}/api/v1/graph/stats")
    if response.status_code == 200:
        stats = response.json()
        print(json.dumps(stats, indent=2))

## 3. TBML Detection

In [ ]:
if api_healthy:
    payload = {"min_depth": 2, "max_depth": 5, "time_window_days": 30}
    response = requests.post(f"{API_BASE_URL}/api/v1/tbml/circular-trading", json=payload)
    if response.status_code == 200:
        results = response.json()
        print(f"Patterns found: {results.get('pattern_count', 0)}")

## 4. Insider Trading Detection

In [ ]:
if api_healthy:
    payload = {"time_window_hours": 48, "min_cluster_size": 2}
    response = requests.post(f"{API_BASE_URL}/api/v1/insider-trading/coordinated", json=payload)
    if response.status_code == 200:
        results = response.json()
        print(f"Clusters found: {results.get('cluster_count', 0)}")

## 5. UBO Discovery

In [ ]:
if api_healthy:
    company_id = "COMP-001"
    response = requests.get(f"{API_BASE_URL}/api/v1/ubo/{company_id}")
    if response.status_code == 200:
        ubo_result = response.json()
        print(f"UBOs found: {len(ubo_result.get('ubos', []))}")

## 6. API Client Class

In [ ]:
class AnalyticsAPIClient:
    def __init__(self, base_url: str = "http://localhost:8000"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def health_check(self) -> bool:
        try:
            return self.session.get(f"{self.base_url}/health").status_code == 200
        except:
            return False
    
    def get_graph_stats(self) -> dict:
        return self.session.get(f"{self.base_url}/api/v1/graph/stats").json()
    
    def detect_circular_trading(self, min_depth=2, max_depth=5) -> dict:
        return self.session.post(
            f"{self.base_url}/api/v1/tbml/circular-trading",
            json={"min_depth": min_depth, "max_depth": max_depth}
        ).json()
    
    def get_ubos(self, company_id: str) -> dict:
        return self.session.get(f"{self.base_url}/api/v1/ubo/{company_id}").json()

client = AnalyticsAPIClient()
print(f"Client ready: {client.health_check()}")

## Summary

This notebook demonstrated:
1. API health checks and connectivity
2. Graph statistics retrieval
3. TBML detection endpoints
4. Insider trading detection
5. UBO discovery API
6. Reusable API client class